# Chainer の学習と推論を SageMaker で行う

MNISTデータセットを対象にSageMakerとChainerを利用してMLPの学習と推論を行います。学習の方法として以下の3種類を試します。
- 単一または複数ノードによる学習: 高性能なトレーニングインスタンスを1つまたは複数立ち上げて学習を行います。
- ハイパーパラメータ最適化: 単一ノードにおける学習でHPO (Hyper-Parameter Optimization: ハイパーパラメータ最適化）を行います。
- ローカルモード: ノートブックインスタンスで学習します。追加のインスタンス立ち上げが不要で、開発時のデバッグに有用です。


## 目次
1. [準備](#準備)
2. [データの取得とS3へのアップロード](#データの取得とS3へのアップロード)
3. [エントリーポイント](#エントリーポイント)
4. [モデルの学習](#モデルの学習)
    1. [単一または複数ノードによる学習](#単一または複数ノードによる学習)
    2. [ハイパーパラメータの最適化](#ハイパーパラメータの最適化)
    3. [ローカルモード](#ローカルモード)
5. [学習結果の可視化](#学習結果の可視化)
6. [ハイパーパラメータのチューニング結果](#ハイパーパラメータのチューニング結果)
7. [モデルの推論を実行](#モデルの推論を実行)
8. [エンドポイントの削除](#エンドポイントの削除)

## 準備

ローカルモードを実行するため、いくつかのパッケージを事前インストールする必要があります。そのためのスクリプト`setup.sh`を用意しているので実行しましょう。

In [ ]:
!sh setup.sh

## データの取得とS3へのアップロード

ここでは、`Chainer` でサポートされている関数を使って、MNIST データをダウンロードします。SageMaker の学習時に利用するデータは、S3 に置く必要があります。ここでは、ローカルにダウンロードした MNIST データを npz 形式で固めてから、SageMaker のラッパー関数を使って S3 にアップロードします。

デフォルトでは SageMaker は `sagemaker-{region}-{your aws account number}` というバケットを使用します。当該バケットがない場合には、自動で新しく作成します。`upload_data()` メソッドの引数に bucket=XXXX という形でデータを配置するバケットを指定することも可能です。

In [ ]:
import chainer
import os
import shutil
import numpy as np

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

# Download MNIST dataset
train, test = chainer.datasets.get_mnist()

# Extract data and labels from dataset 
train_images = np.array([data[0] for data in train])
train_labels = np.array([data[1] for data in train])
test_images = np.array([data[0] for data in test])
test_labels = np.array([data[1] for data in test])

# Save the data and labels as .npz into local directories and upload them to S3
try:
    os.makedirs('/tmp/data/train')
    os.makedirs('/tmp/data/test')

    np.savez('/tmp/data/train/train.npz', images=train_images, labels=train_labels)
    np.savez('/tmp/data/test/test.npz', images=test_images, labels=test_labels)

    train_input = sagemaker_session.upload_data(
        path=os.path.join('/tmp/data', 'train'),
        key_prefix='notebook/chainer/mnist')
    test_input = sagemaker_session.upload_data(
        path=os.path.join('/tmp/data', 'test'),
        key_prefix='notebook/chainer/mnist')
finally:
    shutil.rmtree('/tmp/data')

## エントリーポイント

SageMakerで、Chainer、Tensorflowなどのフレームワークを利用して深層学習を行うためには、このnotebook以外に**エントリーポイントを作成する必要があります**。エントリーポイントとはモデルや学習方法を記述した.pyファイルで、このnotebookには`chainer_mnist.py`というエントリーポイントを同じフォルダに用意しています。ノートブックインスタンスでfit関数を呼び出すと、エントリーポイントに沿って学習が行われます。

chainerを利用する場合は、エントリーポイントの`__main__`関数内にモデルの記述や学習方法を記載すればよく、SageMakerを使う以前のChainerのコードを概ねそのまま利用することができます。また、環境変数経由で入力データの場所や GPU の数などを取得することが可能です。これは `argparse` 経由で `main` 関数内で受け取ることができます。詳細は[こちら](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/chainer/README.rst)をご覧ください。

また推論時の処理は、`model_fn` で学習済みモデルをロードする部分だけ記述する必要があります。その他オプションで、前処理、推論処理、後処理部分を `input_fn`、 `predict_fn`、 `output_fn` で書くこともできます。デフォルトでは、`application/x-npy` コンテントタイプで指定される、NumPy 配列を入力として受け取ります。 

以下のセルを実行してエントリーポイントの中身を表示してみます。すると、`class MLP(chainer.Chain)`といったモデルの定義、`__main__`の中に学習のコードが書かれていることがわかります。また、chainerMNのoptimizerを使用して分散学習を行うようになっています。

In [ ]:
!pygmentize 'chainer_mnist.py'

## モデルの学習

`Estimator` クラスの子クラスの `Chainer` オブジェクトを作成し、`fit()` メソッドで学習ジョブを実行します。 `entry_point` で指定したローカルのスクリプトが、学習用のコンテナ内で実行されます。また合わせてローカルの `source_dir` を指定することで、依存するスクリプト群をコンテナにコピーして、学習時に使用することが可能です。

### 単一または複数ノードによる学習

単一ノードで学習したい場合は、`train_instance_count=1`として、学習用インスタンスを`instance_type`に指定します。複数ノードによる学習は、`train_instance_count`を1より大きくすることで実行できます。複数ノードの場合には、分散学習となるようにエントリーポイントにChainerMNを利用した実装が必要になります。あとで学習の結果を参照するためにジョブの名前を記録しておきます。

In [ ]:
import subprocess

from sagemaker.chainer.estimator import Chainer

instance_type = 'ml.m4.xlarge'

chainer_estimator = Chainer(entry_point='chainer_mnist.py', role=role,
                            train_instance_count=1, train_instance_type=instance_type,
                            framework_version='5.0.0',
                            hyperparameters={'epochs': 3, 'batch-size': 128})

chainer_estimator.fit({'train': train_input, 'test': test_input})

# Keep the job name for checking training loss later 
training_job = chainer_estimator.latest_training_job.name

### ハイパーパラメータの最適化

ハイパーパラメータの最適化は、fitする前に、以下のような処理を書くことによって実行できます。

- ハイパーパラメータの探索条件の設定
    - カテゴリ変数、連続変数、離散変数かどうか、探索範囲の指定を行います。
- ハイパーパラメータを選択する基準（以下では、バリデーションデータに対する精度で選択）
    - エントリーポイント内のPrintReportでバリデーションデータに対する精度を出力 
    - ログからバリデーションデータに対する精度のみを抽出する正規表現を記述  
    
- 上記の探索範囲、選択基準、チューニングのために実行するジョブ数などを指定してHyperparameterTunerを定義

ハンズオンの時間の都合上、以下では最適化のアルゴリズムをSGDかAdamのどちらかを選択するだけのチューニングを行います。
チューニングのジョブを以下のページで確認することができます。`Completed`になるまで5分程度かかりますので、待たずに次のローカルモードに進みましょう。  
https://ap-northeast-1.console.aws.amazon.com/sagemaker/home?region=ap-northeast-1#/hyper-tuning-jobs

In [ ]:
import subprocess
from sagemaker.chainer.estimator import Chainer
instance_type = 'ml.m4.xlarge'
chainer_estimator = Chainer(entry_point='chainer_mnist.py', role=role,
                            train_instance_count=1, train_instance_type=instance_type,
                            framework_version='5.0.0',
                            hyperparameters={'epochs':3, 'batch_size': 128})

###Setting for hyper paramter optimization###
from sagemaker.tuner import HyperparameterTuner,  CategoricalParameter, ContinuousParameter, IntegerParameter

hyperparameter_ranges = {'optimizer': CategoricalParameter(['sgd', 'Adam'])}
'''
An example of further tuning:
hyperparameter_ranges = {'optimizer': CategoricalParameter(['sgd', 'Adam']),
                          'learning_rate': ContinuousParameter(0.01, 0.2),
                          'num_epoch': IntegerParameter(3, 5)}
'''

objective_metric_name = 'Validation-accuracy'
metric_definitions = [{'Name': 'Validation-accuracy',
                       'Regex': 'validation/main/accuracy=([0-9\\.]+)'}]

tuner = HyperparameterTuner(chainer_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=2,
                            max_parallel_jobs=2)
##################################

tuner.fit({'train': train_input, 'test': test_input})

training_job_tuning = tuner.latest_tuning_job.name

### ローカルモード

ノートブックインスタンスのCPUで学習する場合は`instance_type = 'local'`、GPUで学習する場合は`local_gpu`を指定します。インスタンス数は、ノートブックインスタンスの数、すなわち1になるため、 `train_instance_count`に指定された値が1より大きい場合も1として扱われます。

In [ ]:
import subprocess

from sagemaker.chainer.estimator import Chainer

instance_type = 'local'

chainer_estimator = Chainer(entry_point='chainer_mnist.py', role=role,
                            train_instance_count=1, train_instance_type=instance_type,
                            framework_version='5.0.0',
                            hyperparameters={'epochs': 3, 'batch_size': 128})

chainer_estimator.fit({'train': train_input, 'test': test_input})

## ハイパーパラメータのチューニング結果
以下でチューニングのジョブが`complete`になっていることを確認します。チューニングのジョブの表で、先ほど実行したジョブの名前をクリックすると、トレーニングジョブのページに移動します。SGDとAdamのそれぞれの最適化を実行したジョブの結果が表示されており、それぞれの検証スコア（バリデーションデータに対する精度）が表示されていると思います。  
https://ap-northeast-1.console.aws.amazon.com/sagemaker/home?region=ap-northeast-1#/hyper-tuning-jobs  

`describe_training_job`を利用することで、ジョブの詳細を辞書形式で見ることができます。特に、チューニングジョブの詳細を見ることによって、ハイパーパラメータのチューニング結果を知ることができます。例えば、ハイパーパラメータ最適化によって、選択されたOptimizerを知りたい場合は、辞書の\['HyperParameters'\]\['Optimizer'\]を見ます。

In [ ]:
desc = tuner.sagemaker_session.sagemaker_client. \
           describe_training_job(TrainingJobName=tuner.best_training_job())
selected_optimizer = desc['HyperParameters']['optimizer']
print(selected_optimizer)

# モデルの推論を実行


推論を行うために学習したモデルをデプロイします。ここでは、ハイパーパラメータをチューニングした結果からデプロイしましょう。`deploy()` メソッドでは、デプロイ先エンドポイントのインスタンス数、インスタンスタイプを指定します。こちらもインスタンスタイプを `local` にすることで，このインスタンス内にエンドポイントを作成します。

In [ ]:
instance_type = 'ml.m4.xlarge'
predictor = tuner.deploy(initial_instance_count=1, instance_type=instance_type)

デプロイが終わったら実際に手書き文字認識を行ってみましょう。最初はランダムに5枚選んで推論をしてみます。

In [ ]:
%matplotlib inline
import random

import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(test_images.shape[0] - 1), num_samples)
images, labels = test_images[indices], test_labels[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')

prediction = predictor.predict(images)
predicted_label = prediction.argmax(axis=1)
print('The predicted labels are: {}'.format(predicted_label))

次のセルを実行すると、HTMLのcanvasを表示して、枠内に手書きの数字を書くことができます。さらに次のセルを実行すると、キャンバスに書かれた数字に対して、エンドポイントで予測が実行されます。

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

In [ ]:
image = np.array(data, dtype=np.float32)
prediction = predictor.predict(image)
predicted_label = prediction.argmax(axis=1)[0]
print('What you wrote is: {}'.format(predicted_label))

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
tuner.delete_endpoint()